In [2]:
"""
Training Loop
"""
import os
import jax
import jax.numpy as jnp
import numpy as np
import jaxley as jx
from jaxley.solver_gate import solve_gate_exponential, save_exp
from jaxley.channels import Channel
import matplotlib.pyplot as plt
from typing import Dict, Optional, Tuple

# import jaxley.optimize.transforms as jt
# from jax import random
# import tensorflow_probability.substrates.jax as tfp
# import optax
# from jax import jit, vmap, value_and_grad
# import jax.lax as lax

SEED = 0
GPU = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU)

NUM_EPOCHS = 400
SIM_TIME_SAMPLES = 7
TIME_STEP = 2e-3 #ms
TOTAL_SIM_TIME = SIM_TIME_SAMPLES * TIME_STEP #ms

DUMP_PERIOD = 20
TOTAL_LENGTH = 1500.0
SEGMENT_LENGTH = 1.0
assert TOTAL_LENGTH % SEGMENT_LENGTH == 0, "TOTAL_LENGTH must be divisible by SEGMENT_LENGTH"
NUM_COMPARTMENTS = int(TOTAL_LENGTH/SEGMENT_LENGTH)
VOLTAGE_CLAMP_SEGMENT = 50
VOLTAGE_CLAMP_VALUE = 0.0
STOP_CRITERION = 0.05
PERCENTILE_SAMPLE_AND_CLIP = 0.85
SAVE_NAME = f'seed_{SEED}'
np.random.seed(SEED)
ELEC_SPACING = 30.0
ELECTRODE_CONFIGURATION = 'triangle'
ELEC_COORDS = None

if ELECTRODE_CONFIGURATION == 'triangle':
    #place electrodes at equilateral traiangle in x=0 plane, center at (0,0,0), spaced ELEC_SPACING apart
    ELEC_COORDS = np.array([[0, 0, 0], [0, 1, 0], [0, 0.5, 0.866]])
    ELEC_COORDS = ELEC_COORDS - np.mean(ELEC_COORDS, axis=0)
    ELEC_COORDS = ELEC_SPACING*ELEC_COORDS
    ELEC_COORDS = jnp.array(ELEC_COORDS, dtype=jnp.float32)
    ELEC_COORDS = jax.device_put(ELEC_COORDS) # Move to default accelerator (GPU if available)

def _vtrap(x, y):
    return x / (save_exp(x / y) - 1.0)

class HH(Channel):
    """Hodgkin-Huxley channel."""

    def __init__(self, name: Optional[str] = None):
        self.current_is_in_mA_per_cm2 = True

        super().__init__(name)
        prefix = self._name
        self.channel_params = {
            f"{prefix}_gNa": 0.12,
            f"{prefix}_gK": 0.036,
            f"{prefix}_gLeak": 0.0003,
            f"{prefix}_eNa": 60.0,
            f"{prefix}_eK": -77.0,
            f"{prefix}_eLeak": -54.3,
        }
        self.channel_states = {
            f"{prefix}_m": 0.2,
            f"{prefix}_h": 0.2,
            f"{prefix}_n": 0.2,
        }
        self.current_name = f"i_HH"

    def update_states(
        self,
        states: Dict[str, jnp.ndarray],
        dt,
        v,
        params: Dict[str, jnp.ndarray],
    ):
        """Return updated HH channel state."""
        prefix = self._name
        m, h, n = states[f"{prefix}_m"], states[f"{prefix}_h"], states[f"{prefix}_n"]
        new_m = solve_gate_exponential(m, dt, *self.m_gate(v))
        new_h = solve_gate_exponential(h, dt, *self.h_gate(v))
        new_n = solve_gate_exponential(n, dt, *self.n_gate(v))
        return {f"{prefix}_m": new_m, f"{prefix}_h": new_h, f"{prefix}_n": new_n}

    def compute_current(
        self, states: Dict[str, jnp.ndarray], v, params: Dict[str, jnp.ndarray]
    ):
        """Return current through HH channels."""
        prefix = self._name
        m, h, n = states[f"{prefix}_m"], states[f"{prefix}_h"], states[f"{prefix}_n"]

        gNa = params[f"{prefix}_gNa"] * (m**3) * h  # S/cm^2
        gK = params[f"{prefix}_gK"] * n**4  # S/cm^2
        gLeak = params[f"{prefix}_gLeak"]  # S/cm^2

        return (
            gNa * (v - params[f"{prefix}_eNa"])
            + gK * (v - params[f"{prefix}_eK"])
            + gLeak * (v - params[f"{prefix}_eLeak"])
        )

    def init_state(self, states, v, params, delta_t):
        """Initialize the state such at fixed point of gate dynamics."""
        prefix = self._name
        alpha_m, beta_m = self.m_gate(v)
        alpha_h, beta_h = self.h_gate(v)
        alpha_n, beta_n = self.n_gate(v)
        return {
            f"{prefix}_m": alpha_m / (alpha_m + beta_m),
            f"{prefix}_h": alpha_h / (alpha_h + beta_h),
            f"{prefix}_n": alpha_n / (alpha_n + beta_n),
        }

    @staticmethod
    def m_gate(v):
        alpha = 2.725 * _vtrap(-(v + 35), 10)
        beta = 90.83 * save_exp(-(v + 60) / 20)
        return alpha, beta

    @staticmethod
    def h_gate(v):
        alpha = 1.817 * save_exp(-(v + 52) / 20)
        beta = 27.25 / (save_exp(-(v + 22) / 10) + 1)
        return alpha, beta

    @staticmethod
    def n_gate(v):
        alpha = 0.09575 * _vtrap(-(v + 37), 10)
        beta = 1.915 * save_exp(-(v + 47) / 80)
        return alpha, beta

def transform_point(x: jnp.ndarray, y: jnp.ndarray, z: jnp.ndarray, 
                   phi: jnp.ndarray, theta: jnp.ndarray) -> Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """
    Transform a point using phi and theta rotations.
    
    """
    final_z = z * jnp.cos(phi) - x * jnp.sin(phi)
    int_x = x * jnp.cos(phi) + z * jnp.sin(phi)
    final_x = int_x * jnp.cos(theta) - y * jnp.sin(theta)
    final_y = y * jnp.cos(theta) + int_x * jnp.sin(theta)
    return final_x, final_y, final_z

def compute_cell_locations_from_orientations(
    axon_origin_dist: jnp.ndarray,
    phi: jnp.ndarray, 
    theta: jnp.ndarray,
    axon_spin_angle: jnp.ndarray
) -> jnp.ndarray:
    """
    Compute cell compartment locations from orientation parameters.
    """
    # Calculate original endpoints before phi and theta rotations using the spin angle
    original_first_x = -1 * TOTAL_LENGTH / 2 * jnp.cos(axon_spin_angle)
    original_first_y = -1 * TOTAL_LENGTH / 2 * jnp.sin(axon_spin_angle)
    original_first_z = axon_origin_dist
    
    # Transform the endpoints
    ff_x, ff_y, ff_z = transform_point(original_first_x, original_first_y, original_first_z, phi, theta)
    fl_x, fl_y, fl_z = transform_point(-1 * original_first_x, -1 * original_first_y, original_first_z, phi, theta)

    #return stacked interpolation between x,y,z endpoints in one tensor
    return jnp.stack([
        jnp.linspace(ff_x, fl_x, NUM_COMPARTMENTS, endpoint=False),
        jnp.linspace(ff_y, fl_y, NUM_COMPARTMENTS, endpoint=False),
        jnp.linspace(ff_z, fl_z, NUM_COMPARTMENTS, endpoint=False)
    ], axis=1)


class SimpleCellEIAndLoss:
    def __init__(self):
        comp = jx.Compartment()
        branch = jx.Branch([comp]*NUM_COMPARTMENTS)
        self.cell = jx.Cell([branch], parents=[-1])
        self.cell.insert(HH())
        self.cell.set("HH_gLeak", 1e-4)    # Leak conductance in S/cm^2
        self.cell.set("HH_eNa", 60.60)     # Sodium reversal potential in mV
        self.cell.set("HH_eK", -101.34)     # Potassium reversal potential in mV
        self.cell.set("HH_eLeak", -64.58)  # Leak reversal potential in mV
        self.cell.set("HH_m", 0.0353)        # Initial value of m gate
        self.cell.set("HH_h", 0.9054)        # Initial value of h gate  
        self.cell.set("HH_n", 0.0677)        # Initial value of n gate
        self.cell.set("length", SEGMENT_LENGTH)  
        # Parameter bounds (same as PyTorch version)
        self.MODEL_BOUNDS = {
            'axon_origin_dist': (10.0, 30.0),
            'axon_theta': (-jnp.pi/2, jnp.pi/2),
            'axon_phi': (0.0, jnp.pi),
            'axon_spin_angle': (-jnp.pi/2, jnp.pi/2),
            'fiber_radius_um': (1.0, 5.0),
            'sodium_channel_density': (0.1, 0.3),
            'potassium_channel_density': (0.1, 0.3),
            'axial_resistivity': (50.0, 200.0)
        }
        # Parameters to optimize (same as PyTorch version)
        self.params_list = [
            'axon_origin_dist', 'axon_theta', 
            'axon_phi', 'axon_spin_angle',
            'fiber_radius_um', 'sodium_channel_density',
            'potassium_channel_density', 'axial_resistivity'
        ]
        # Initialize raw parameters (unconstrained, for optimization)
        self.raw_params = self._initialize_raw_params_midpoint()
        self.locations = jnp.zeros((NUM_COMPARTMENTS, 3))  # Initialize empty locations tensor

    def _convert_to_raw(self, param_val, min_val, max_val):
        """Convert constrained parameter to raw (unconstrained) space."""
        ratio = (param_val - min_val) / (max_val - min_val)
        # Clamp to avoid logit of 0 or 1
        ratio = jnp.clip(ratio, 0.001, 0.999)
        return jnp.log(ratio / (1.0 - ratio))  # logit, inverse of sigmoid

    def _sigmoid_reparametrize(self, raw_val, min_val, max_val):
        """Convert raw parameter back to constrained space using sigmoid."""
        ratio = 1.0 / (1.0 + jnp.exp(-raw_val))  # sigmoid
        return min_val + ratio * (max_val - min_val)

    def _initialize_raw_params_midpoint(self):
        """Initialize raw parameters with midpoint values."""
        raw_params = {}
        for param_name in self.params_list:
            bounds = self.MODEL_BOUNDS[param_name]
            # Start at midpoint of bounds
            midpoint = (bounds[0] + bounds[1]) / 2.0
            # Convert to raw (unconstrained) space
            raw_val = self._convert_to_raw(midpoint, bounds[0], bounds[1])
            raw_params[param_name] = jnp.array(raw_val)
        return raw_params
    
    def set_params(self, params: Dict[str, jnp.ndarray]):
        """Set parameters from a dictionary."""
        for param_name, value in params.items():
            self.raw_params[param_name] = self._convert_to_raw(
                value, self.MODEL_BOUNDS[param_name][0], self.MODEL_BOUNDS[param_name][1])



    def get_parameters(self):
        """Convert raw parameters to constrained parameters."""
        transformed_params = {}
        for param_name in self.params_list:
            bounds = self.MODEL_BOUNDS[param_name]
            raw_val = self.raw_params[param_name]
            transformed_val = self._sigmoid_reparametrize(raw_val, bounds[0], bounds[1])
            transformed_params[param_name] = transformed_val
        return transformed_params


    def transform_params(self):
        """Update cell parameters based on current transformed parameters."""

        transformed_params = self.get_parameters()

        self.cell.set("HH_gNa", transformed_params['sodium_channel_density'])
        self.cell.set("HH_gK", transformed_params['potassium_channel_density'])
        self.cell.set("axial_resistivity", transformed_params['axial_resistivity'])
        self.cell.set("radius", transformed_params['fiber_radius_um'])

        # Update cell morphology using our JAX function
        self.locations = compute_cell_locations_from_orientations(
            axon_origin_dist=transformed_params['axon_origin_dist'],
            phi=transformed_params['axon_phi'],
            theta=transformed_params['axon_theta'],
            axon_spin_angle=transformed_params['axon_spin_angle']
        )

        self.cell.set("x", self.locations[:, 0])
        self.cell.set("y", self.locations[:, 1])
        self.cell.set("z", self.locations[:, 2])

    def compute_membrane_currents(self):
        """Compute membrane currents using current parameters."""
        # This would be implemented based on your specific needs
        self.transform_params()
        rest_voltage = self.cell.nodes["v"][0]
        print(f"rest_voltage: {rest_voltage}")
        #set the starting voltage for the first CLAMP compartments to the clamp voltage
        voltage_vector = jnp.array([VOLTAGE_CLAMP_VALUE] * VOLTAGE_CLAMP_SEGMENT + 
                                   [rest_voltage] * (NUM_COMPARTMENTS - VOLTAGE_CLAMP_SEGMENT))
        self.cell.set("v", voltage_vector)

        current = jx.step_current(i_delay=1.0, i_dur=1.0, i_amp=0.0, delta_t=TIME_STEP, t_max=TOTAL_SIM_TIME)
        self.cell.branch(0).loc(0.0).stimulate(current)

        # # Record total transmembrane current
        self.cell.record("v")
        #self.cell.record("i_mem")
        #self.cell.record("HH_m")  # sodium activation
        #self.cell.record("HH_h")  # sodium inactivation 
        #self.cell.record("HH_n")  # potassium activation
        
        # # Run simulation
        #return jx.integrate(self.cell, delta_t=TIME_STEP)



    def compute_eap(self):
        """Compute extracellular action potentials."""
        # This would be implemented based on your specific needs
        pass

cell_trainer = SimpleCellEIAndLoss()
cell_trainer.transform_params()

# Convert cell nodes to dataframe
import pandas as pd
# Convert cell nodes to dataframe and save as CSV
nodes_df = pd.DataFrame(cell_trainer.cell.nodes)
nodes_df.to_csv('/pool0/lotlikar/software/extracellular-jaxley/src/cell_nodes.csv', index=False)

outputs = cell_trainer.compute_membrane_currents()



rest_voltage: -70.0
Added 1 external_states. See `.externals` for details.
Added 1500 recordings. See `.recordings` for details.


In [16]:
print(outputs.shape)
# Reshape outputs to (4, 100, ...) for the 4 recorded variables over 100 timesteps
reshaped_outputs = outputs.reshape(4, 100, -1)
print("Reshaped outputs shape:", reshaped_outputs.shape)

print(reshaped_outputs[2, 0, :])

(400, 79)
Reshaped outputs shape: (4, 100, 79)
[0.9054     0.86208063 0.82200843 0.785811   0.7532714  0.7235171
 0.6955084  0.6683768  0.6415399  0.614702   0.5878066  0.5609681
 0.5343943  0.5083169  0.48294246 0.45842806 0.43487674 0.4123445
 0.39085147 0.37039277 0.3509471  0.3324832  0.31496397 0.2983495
 0.28259876 0.26767072 0.25352517 0.24012294 0.22742635 0.21539925
 0.20400704 0.19321679 0.18299711 0.1733182  0.16415174 0.15547085
 0.14725003 0.13946514 0.13209325 0.12511268 0.11850291 0.11224447
 0.10631897 0.10070898 0.09539807 0.09037066 0.08561207 0.08110843
 0.07684668 0.07281452 0.06900038 0.06539343 0.0619835  0.05876115
 0.05571759 0.05284471 0.05013508 0.04758194 0.04517924 0.04292165
 0.04080456 0.03882414 0.03697735 0.03526196 0.03367659 0.03222069
 0.03089455 0.02969921 0.02863639 0.02770837 0.02691779 0.02626745
 0.02576013 0.02539835 0.02518423 0.02511953 0.02520556 0.02544333
 0.02583371]


In [ ]:
class JaxleyTrainer:
    def __init__(self, cell, data, epochs, dt = 0.05, tmax = 5 ):
        # load through data
        self.cell = cell
        self.data_point = data
        
        self.i_delay = 1.0  # ms
        self.i_dur = 0.15  # ms
        self.i_amp = 3.9  # nA
        self.electrode = generate_electrode_map(519)
        self.ncomp = self.cell.shape[1]
        self.params = self.setup_params()
        for i, param_dict in enumerate(self.params):
            for param_name, param_values in param_dict.items():
                if param_name in ['radius', 'length']:
                    print(f"Branch {i}, {param_name}: shape={param_values.shape}")
                    print(f"  min={jnp.min(param_values):.10f}")
                    print(f"  max={jnp.max(param_values):.10f}")
                    print(f"  mean={jnp.mean(param_values):.10f}")
                    print(f"  any_nan={jnp.any(jnp.isnan(param_values))}")
                    print(f"  any_inf={jnp.any(jnp.isinf(param_values))}")
        transforms = [{k: self.create_transforms(k) for k in param} for param in self.params]

        self.tf = jt.ParamTransform(transforms)
        self.lengths = self.cell.nodes["length"] ### fix
        self.lengths.to_numpy()
        self.lengths = jnp.array(self.lengths)
        self.N_ELECTRODES = LITKE_519_ARRAY_GRID.shape[0]
        self.grid = jnp.array(LITKE_519_ARRAY_GRID)
        self.delta_t = 0.05 # ms
        self.t_max = 5
        
        self.num_epochs = epochs
        self.current_compartment_positions = fh.compute_comp_xyz(self.cell)
        self.compartment_surface_areas = fh.get_surface_areas(self.cell) * 10E-8  
        self.opt_params = self.tf.inverse(self.params)

        hold_steps = 15000  # Number of steps to hold the learning rate constant
        decay_steps = 30000  # Total training steps (or however many steps you want the cosine decay to complete)

        # 1. Constant learning rate for first 5000 steps
        constant = optax.constant_schedule(value=1e-1)

        # 2. Cosine decay from 1e-1 to 1e-5
        cosine = optax.cosine_decay_schedule(
            init_value=1e-1,
            decay_steps=decay_steps - hold_steps,
            alpha=1e-5 / 1e-1  # final_lr / init_lr
        )

        # Combine the schedules
        schedule = optax.join_schedules(
            schedules=[constant, cosine],
            boundaries=[hold_steps]
        )
        self.optimizer = optax.chain(
            optax.clip_by_global_norm(1.0), 
            optax.adam(learning_rate=schedule)
        )
        self.opt_state = self.optimizer.init(self.opt_params)
        
        self.jitted_grad = jit(value_and_grad(self.loss, argnums=0))

    def compute_comp_xyz(self, transformed_params):
        '''
        Returns (Ncomps, 3) array of xyz positions of all the compartments
        '''
        
        # 
        all_x = []
        all_y = []
        all_z = []
    
        for param_dict in transformed_params:
            if 'x' in param_dict:
                all_x.append(param_dict['x'])
            if 'y' in param_dict:
                all_y.append(param_dict['y'])
            if 'z' in param_dict:
                all_z.append(param_dict['z'])
        xs = jnp.concatenate(all_x) 
        ys = jnp.concatenate(all_y) 
        zs = jnp.concatenate(all_z) 
        
        comp_xyz = jnp.stack([xs, ys, zs], axis=1)
        return comp_xyz
    
    def final_compute_xyz(self, transformed_params):
        all_x = []
        all_y = []
        all_z = []
    
        for param_dict in transformed_params:
            if 'x' in param_dict:
                all_x.append(param_dict['x'])
            if 'y' in param_dict:
                all_y.append(param_dict['y'])
            if 'z' in param_dict:
                all_z.append(param_dict['z'])
        xs = jnp.concatenate(all_x) 
        ys = jnp.concatenate(all_y) 
        zs = jnp.concatenate(all_z) 
        return xs, ys, zs
    def get_surface_areas(self, transformed_params):
        '''
        Returns (Ncomps,) array of the surface areas in um2 of all the compartments
        JAX-compatible version
        '''
        all_radii = []
        all_lengths = []
        
        for param_dict in transformed_params:
            if 'radius' in param_dict:
                all_radii.append(param_dict['radius'])
            if 'length' in param_dict:
                all_lengths.append(param_dict['length'])

        radii = jnp.concatenate(all_radii) 
        lengths = jnp.concatenate(all_lengths) 

        
        surf_areas = 2 * jnp.pi * radii * lengths
        return surf_areas
    
    def create_transforms(self, name: str) -> jt.Transform:
        """
        Creates robust parameter transformations using SigmoidTransform
        for physical constraints.
        """
        # --- For morphology coordinates ---
        if name in ['x', 'y', 'z']:
            return jt.AffineTransform(1.0, 0.0)  # No constraints
        if name in ['radius', 'length']:
            return jt.ChainTransform([jt.SoftplusTransform(0), jt.AffineTransform(10, 0)])
        
        return jt.AffineTransform(1.0, 0.0)
        
        
    def setup_params(self):
        self.cell.delete_trainables()
        self.cell.comp("all").make_trainable("x")
        self.cell.comp("all").make_trainable("y")
        self.cell.comp("all").make_trainable("z")
        # self.cell.comp("all").make_trainable("radius")
        # self.cell.comp("all").make_trainable("length")
        # self.cell.comp("all").make_trainable("HH_eNa")
        # self.cell.comp("all").make_trainable("HH_eK")
        # self.cell.comp("all").make_trainable('axial_resistivity')
        # self.cell.comp("all").make_trainable('HH_gNa')
        # self.cell.comp("all").make_trainable('HH_gLeak')
        # self.cell.comp("all").make_trainable('gCa')
        # self.cell.comp("all").make_trainable('HH_gK')
        # self.cell.comp("all").make_trainable('gKCa')

        return self.cell.get_parameters()


    def simulate(self, params):
        """
        
        """
        
        
        
        delta_t = 0.05 # ms
        t_max = 5
        time_vec = np.arange(0, t_max+2*delta_t, delta_t)

        i_delay = 1.0 
        i_dur = 0.5
        i_amp = 15
        current = jx.step_current(i_delay=i_delay, i_dur=i_dur, i_amp=i_amp, delta_t=delta_t, t_max=t_max)

        self.cell.delete_stimuli()
        data_stimuli = None
        data_stimuli = self.cell.branch(0).comp(0).data_stimulate(current)

        self.cell.delete_recordings()
        self.cell.record("v")
        self.cell.record("i_HH")
        self.cell.record("i_Ca")
        self.cell.record("i_KCa")

        sim_outputs = jx.integrate(self.cell, params = params, data_stimuli=data_stimuli).reshape(4, self.ncomp, -1)
        return sim_outputs
    
    def gen_ei(self, outputs, transformed_params):
        """
        Generate extracellular potential from simulation outputs
        """
        tfd = tfp.distributions
        seed = random.PRNGKey(0)

        time_vec = np.arange(0, self.t_max+2*self.delta_t, self.delta_t)
        current_compartment_positions = self.compute_comp_xyz(transformed_params)
        # compartment_surface_areas = self.get_surface_areas(transformed_params) * 10E-8  # Calculate surface areas in cm^2
        
        current_distance = fh.distance(LITKE_519_ARRAY_GRID, current_compartment_positions) * 10E-4 
        
        sim_v_extra = fh.compute_eap(outputs, current_distance, self.compartment_surface_areas) \
             + tfd.Normal(0, 0.0001).sample((self.N_ELECTRODES, len(time_vec)), seed=seed)
        sim_EI = self.with_ttl(sim_v_extra).T
        return sim_EI
    
    def with_ttl(self, raw_data):
        shape_array = jnp.array(raw_data.shape)
        has_512 = jnp.any(shape_array == 512)
        has_519 = jnp.any(shape_array == 519)
  
        critical_axis_512 = jnp.where(shape_array == 512, size=1, fill_value=-1)[0]
        critical_axis_519 = jnp.where(shape_array == 519, size=1, fill_value=-1)[0]
        
        critical_axis = jnp.where(has_512, critical_axis_512, critical_axis_519)

        append_data_shape = list(raw_data.shape)
        
        for i, dim_size in enumerate(raw_data.shape):
            if dim_size == 512 or dim_size == 519:
                critical_axis = i
                break
        
        append_data_shape[critical_axis] = 1
        append_data = jnp.zeros(append_data_shape)
        
        return jnp.concatenate((append_data, raw_data), axis=critical_axis)

    def predict(self, params):
        outputs = self.simulate(params)
        predicted_ei = self.gen_ei(outputs, params)
        return predicted_ei

    def loss(self, opt_params, true_ei):
        """
        Computes a loss that is robust to temporal shifts by using a
        differentiable soft-alignment mechanism.
        """
        transformed_params = self.tf.forward(opt_params)
        predicted_ei = self.predict(transformed_params)

        true_len = true_ei.shape[0]
        pred_len = predicted_ei.shape[0]
        n_electrodes = true_ei.shape[1]
        max_offset = pred_len - true_len

        def compute_loss_at_offset(offset):
            pred_window = lax.dynamic_slice(
                predicted_ei,
                start_indices=[offset, 0],
                slice_sizes=[true_len, n_electrodes]
            )

            epsilon = 1e-6
            pred_norm = (pred_window - jnp.mean(pred_window)) / (jnp.std(pred_window) + epsilon)
            true_norm = (true_ei - jnp.mean(true_ei)) / (jnp.std(true_ei) + epsilon)

            mse = jnp.mean((pred_norm - true_norm)**2)
            return mse

        offsets = jnp.arange(max_offset + 1)
        all_mse_losses = vmap(compute_loss_at_offset)(offsets)

        
        temperature = 1.0
        weights = jax.nn.softmax(-all_mse_losses * temperature)

        final_loss = jnp.sum(weights * all_mse_losses)


        return final_loss

    
    def train(self):
        epoch_losses = []
        for epoch in range(self.num_epochs):
            epoch_loss = 0.0
            
            loss_val, gradient = self.jitted_grad(self.opt_params, self.data_point)
            flat_grads, _ = jax.tree_util.tree_flatten(gradient)
            if jnp.isnan(loss_val):
                break
            if flat_grads: 
                grad_norm = jnp.linalg.norm(jnp.concatenate([g.flatten() for g in flat_grads]))
                print(f"Epoch {epoch}, Loss {loss_val}, Grad Norm: {grad_norm}")
            else:
                print(f"Epoch {epoch}, Loss {loss_val}, No gradients found.")
            updates, self.opt_state = self.optimizer.update(gradient, self.opt_state)
            self.opt_params = optax.apply_updates(self.opt_params, updates)
            epoch_loss += loss_val
        
            print(f"epoch {epoch}, loss {epoch_loss}")
            epoch_losses.append(epoch_loss)
        final_params = self.tf.forward(self.opt_params)
        # xs,ys,zs = self.final_compute_xyz(final_params) 
        # self.cell.comp(all).move_to(xs,ys,zs, update_nodes=True)




        sim_ei = self.predict(final_params)
        return final_params, sim_ei, epoch_losses